In [1]:
import dataset
import tensorflow as tf
import pandas as pd
import numpy as np
import csv

TRAINING_SET_FRACTION = 0.95

In [2]:
data = pd.read_csv('data/data.csv')
display(data)


,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,BbAv<2.5,BbAH,BbAHh,BbMxAHH,BbAvAHH,BbMxAHA,BbAvAHA,Unnamed: 56,Unnamed: 57,Unnamed: 58
0,E0,16/08/08,Arsenal,West Brom,1,0,H,1,0,H,...,2.14,26.0,-1.5,1.88,1.81,2.12,1.97,NaN,NaN,NaN
1,E0,16/08/08,Bolton,Stoke,3,1,H,3,0,H,...,1.64,27.0,0.0,1.35,1.31,3.56,3.27,NaN,NaN,NaN
2,E0,16/08/08,Everton,Blackburn,2,3,A,1,1,D,...,1.63,28.0,0.0,1.73,1.43,3.05,2.76,NaN,NaN,NaN
3,E0,16/08/08,Hull,Fulham,2,1,H,1,1,D,...,1.64,26.0,0.0,1.91,1.85,2.07,1.94,NaN,NaN,NaN
4,E0,16/08/08,Middlesbrough,Tottenham,2,1,H,0,0,D,...,1.92,27.0,0.0,2.45,2.30,1.65,1.58,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3859,E0,12/05/19,Liverpool,Wolves,2,0,H,1,0,H,...,2.31,22.0,-1.5,1.98,1.91,2.01,1.95,NaN,NaN,NaN
3860,E0,12/05/19,Man United,Cardiff,0,2,A,0,1,A,...,2.95,21.0,-2.0,2.52,2.32,1.72,1.64,NaN,NaN,NaN
3861,E0,12/05/19,Southampton,Huddersfield,1,1,D,1,0,H,...,2.29,22.0,-1.5,2.27,2.16,1.80,1.73,NaN,NaN,NaN
3862,E0,12/05/19,Tottenham,Everton,2,2,D,1,0,H,...,2.07,19.0,-0.5,2.13,2.08,1.85,1.80,NaN,NaN,NaN


In [3]:
data = dataset.Dataset('data/data.csv')

train_results_len = int(TRAINING_SET_FRACTION * len(data.processed_results))
train_results = data.processed_results[:train_results_len]
test_results = data.processed_results[train_results_len:]

In [4]:
display (train_results)

[{'result': 'H',
  'odds-home': 1.17,
  'odds-draw': 6.5,
  'odds-away': 21.0,
  'home-wins': 7,
  'home-draws': 2,
  'home-losses': 1,
  'home-goals': 22,
  'home-opposition-goals': 4,
  'home-shots': 178,
  'home-shots-on-target': 92,
  'home-opposition-shots': 92,
  'home-opposition-shots-on-target': 52,
  'away-wins': 3,
  'away-draws': 3,
  'away-losses': 4,
  'away-goals': 9,
  'away-opposition-goals': 11,
  'away-shots': 105,
  'away-shots-on-target': 52,
  'away-opposition-shots': 124,
  'away-opposition-shots-on-target': 81},
 {'result': 'D',
  'odds-home': 2.0,
  'odds-draw': 3.3,
  'odds-away': 4.0,
  'home-wins': 4,
  'home-draws': 1,
  'home-losses': 5,
  'home-goals': 10,
  'home-opposition-goals': 15,
  'home-shots': 110,
  'home-shots-on-target': 64,
  'home-opposition-shots': 151,
  'home-opposition-shots-on-target': 83,
  'away-wins': 4,
  'away-draws': 0,
  'away-losses': 6,
  'away-goals': 14,
  'away-opposition-goals': 18,
  'away-shots': 116,
  'away-shots-on-targ

In [5]:
def map_results(results):
    features = {}

    for result in results:
        for key in result.keys():
            if key not in features:
                features[key] = []

            features[key].append(result[key])

    for key in features.keys():
        features[key] = np.array(features[key])

    return features, features['result']

In [6]:
train_features, train_labels = map_results(train_results)
test_features, test_labels = map_results(test_results)

In [7]:
display (train_features)

{'result': array(['H', 'D', 'A', ..., 'H', 'H', 'D'], dtype='<U1'),
 'odds-home': array([1.17, 2.  , 1.91, ..., 2.25, 1.19, 2.2 ]),
 'odds-draw': array([6.5 , 3.3 , 3.4 , ..., 3.25, 6.5 , 3.2 ]),
 'odds-away': array([21.  ,  4.  ,  4.2 , ...,  3.2 , 13.  ,  3.39]),
 'home-wins': array([7, 4, 4, ..., 3, 7, 2]),
 'home-draws': array([2, 1, 2, ..., 3, 2, 2]),
 'home-losses': array([1, 5, 4, ..., 4, 1, 6]),
 'home-goals': array([22, 10, 10, ...,  9, 26,  7]),
 'home-opposition-goals': array([ 4, 15, 15, ..., 16,  8, 17]),
 'home-shots': array([178, 110, 120, ...,  85, 182, 113]),
 'home-shots-on-target': array([92, 64, 68, ..., 25, 73, 43]),
 'home-opposition-shots': array([ 92, 151, 130, ..., 164,  75, 147]),
 'home-opposition-shots-on-target': array([52, 83, 69, ..., 54, 28, 64]),
 'away-wins': array([3, 4, 2, ..., 4, 3, 2]),
 'away-draws': array([3, 0, 2, ..., 4, 3, 4]),
 'away-losses': array([4, 6, 6, ..., 2, 4, 4]),
 'away-goals': array([ 9, 14, 11, ..., 14, 15,  7]),
 'away-oppositio

In [8]:
#test_data.columns

In [9]:
import pandas as pd
X_train = pd.DataFrame(train_features)
X_train.drop(columns = ["result"], inplace=True)
Y_train = pd.DataFrame(train_labels,columns=['result'])
Y_train.replace({"result":{'H':1,'A':2,'D':0}},inplace=True)

In [10]:
print (X_train)
print (type(X_train))

      odds-home  odds-draw  odds-away  home-wins  home-draws  home-losses  \
0          1.17       6.50      21.00          7           2            1   
1          2.00       3.30       4.00          4           1            5   
2          1.91       3.40       4.20          4           2            4   
3          8.00       4.50       1.40          3           1            6   
4          3.50       3.30       2.10          1           3            6   
...         ...        ...        ...        ...         ...          ...   
3420       2.10       3.75       3.60          6           3            1   
3421       6.25       4.00       1.61          1           2            7   
3422       2.25       3.25       3.20          3           3            4   
3423       1.19       6.50      13.00          7           2            1   
3424       2.20       3.20       3.39          2           2            6   

      home-goals  home-opposition-goals  home-shots  home-shots-on-target  

In [11]:
X_test = pd.DataFrame(test_features)
X_test.drop(columns = ["result"], inplace=True)
Y_test = pd.DataFrame(test_labels,columns=['result'])
Y_test.replace({"result":{'H':1,'A':2,'D':0}},inplace=True)


In [12]:
print(Y_train)

      result
0          1
1          0
2          2
3          1
4          1
...      ...
3420       0
3421       1
3422       1
3423       1
3424       0

[3425 rows x 1 columns]


In [13]:
from keras.utils import to_categorical
Y_train = to_categorical(Y_train)
Y_test = to_categorical(Y_test)

display(Y_train)

Using TensorFlow backend.


array([[0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       ...,
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.]], dtype=float32)

In [14]:
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD,Adagrad
from keras import regularizers
from keras.callbacks import EarlyStopping

In [15]:
model = Sequential([
    Dense(59,input_shape=(21,),activation='relu',),
    Dense(80,activation='relu',kernel_regularizer=regularizers.l1(0.001)),
    Dense(500,activation='tanh',kernel_regularizer=regularizers.l1(0.001)),
    Dense(100,activation='relu',kernel_regularizer=regularizers.l1(0.001)),
    Dense(3,activation='softmax',kernel_regularizer=regularizers.l1(0.001)),
])
model.compile(optimizer=Adagrad(lr=0.1),loss='categorical_crossentropy',metrics=['acc'])

In [16]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 59)                1298      
_________________________________________________________________
dense_2 (Dense)              (None, 80)                4800      
_________________________________________________________________
dense_3 (Dense)              (None, 500)               40500     
_________________________________________________________________
dense_4 (Dense)              (None, 100)               50100     
_________________________________________________________________
dense_5 (Dense)              (None, 3)                 303       
Total params: 97,001
Trainable params: 97,001
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(x=X_train,y=Y_train,batch_size=64,epochs=1000,validation_split=0.2,
        #callbacks=[EarlyStopping(monitor='val_loss',patience=0.001)]
         )

In [ ]:
model.evaluate(x=X_test,y=Y_test,batch_size=64)